# Text Extract in Images

## Imports


In [31]:
import requests
from PIL import Image
import pytesseract
from io import BytesIO
import json
import time as TIME

## Load data


In [32]:
pages_1st_process = []
pages_2nd_process = []
pages_fact_checking = []

with open("./image_model/json_data_fast_check_1_fixed.json", 'r', encoding='utf-8') as file:
    pages_1st_process = json.load(file)  

with open("./image_model/json_data_fast_check_2_fixed.json", 'r', encoding='utf-8') as file:
    pages_2nd_process = json.load(file)  

with open("./image_model/json_data_fact_checking.json", 'r', encoding='utf-8') as file:
    pages_fact_checking = json.load(file)  

## Text Extracting


In [33]:
def extract_text(image): 
    start = TIME.time()
    text = pytesseract.image_to_string(image, lang='spa') 
    end = TIME.time()
    return text, end - start

def extract_text_from_pages(pages):
    texts_in_images = {"pages": []}
    for index, page in enumerate(pages):
        texts_in_images["pages"].append(page)
        for img_index, image in enumerate(page["images"]):
            try: 
                response = None
                img = None
                try: 
                    response = requests.get(image)
                    img = Image.open(BytesIO(response.content))
                except: 
                    print(f"[0.00 s] - [p: {index:02}][i: {img_index:02}] Ocurrio un error al obtener la imagen [{image}]")
                    continue
                text, time = extract_text(img)
                texts_in_images["pages"][index]["images"][img_index] = {"image": image, "text": text, "error": False}
                print(f"[{time:.2f} s] - [p: {index:02}][i: {img_index:02}] Éxito [{len(text)} caracteres]")
            except Exception as e: 
                texts_in_images["pages"][index]["images"][img_index] = {"image": image, "text": "", "error": True}
                print(f"[{time:.2f} s] - [p: {index:02}][i: {img_index:02}] Ocurrio un error al analizar la imagen [{e}]")
        # if index == 2:
        #     break
        page_text = texts_in_images["pages"][index]["text"]
        page_text = page_text + " ".join([img["text"] for img in texts_in_images["pages"][index]["images"] if not img["error"]])
        texts_in_images["pages"][index]["text"] = page_text
    return texts_in_images

In [34]:
pages_1st_process = extract_text_from_pages(pages_1st_process["pages"])
pages_2nd_process = extract_text_from_pages(pages_2nd_process["pages"])
pages_fact_checking = extract_text_from_pages(pages_fact_checking["pages"])

[0.86 s] - [p: 00][i: 00] Éxito [191 caracteres]
[0.77 s] - [p: 00][i: 01] Éxito [250 caracteres]
[0.93 s] - [p: 00][i: 02] Éxito [1007 caracteres]
[0.94 s] - [p: 03][i: 00] Éxito [762 caracteres]
[0.92 s] - [p: 03][i: 01] Éxito [642 caracteres]
[0.34 s] - [p: 03][i: 02] Éxito [302 caracteres]
[0.39 s] - [p: 04][i: 00] Éxito [66 caracteres]
[0.90 s] - [p: 04][i: 01] Éxito [959 caracteres]
[0.49 s] - [p: 05][i: 00] Éxito [211 caracteres]
[0.26 s] - [p: 05][i: 01] Éxito [0 caracteres]
[0.58 s] - [p: 07][i: 00] Éxito [193 caracteres]
[0.61 s] - [p: 07][i: 01] Éxito [0 caracteres]
[0.63 s] - [p: 09][i: 00] Éxito [124 caracteres]
[0.30 s] - [p: 10][i: 00] Éxito [56 caracteres]
[0.88 s] - [p: 10][i: 01] Éxito [615 caracteres]
[0.74 s] - [p: 10][i: 02] Éxito [609 caracteres]
[0.18 s] - [p: 11][i: 00] Éxito [0 caracteres]
[0.26 s] - [p: 11][i: 01] Éxito [0 caracteres]
[0.18 s] - [p: 11][i: 02] Éxito [0 caracteres]
[0.19 s] - [p: 11][i: 03] Éxito [0 caracteres]
[0.19 s] - [p: 11][i: 04] Éxito [

# Statistics


In [35]:
pages = {"pages": []}
pages["pages"] = pages_1st_process["pages"] + pages_2nd_process["pages"] + pages_fact_checking["pages"]

sum = 0
for page in pages["pages"]:
    sum += len(page["text"])
print(f"Cantidad de Texto: {sum}")

sum = 0
for page in pages["pages"]:
    for image in page["images"]:
        sum += len(image["text"])
print(f"Texto adicional en imagenes: {sum}")

print("")

print(f"Total de imagenes {len([image for page in pages['pages'] for image in page['images']])}")
print(f"Imagenes con texto adicional: {len([image for page in pages['pages'] for image in page['images'] if len(image['text']) > 0])}")

Cantidad de Texto: 1188243
Texto adicional en imagenes: 58686

Total de imagenes 340
Imagenes con texto adicional: 244


# Save data


In [ ]:
with open(f"./image_model/json_data_fast_check_1_fixed.json", 'w', encoding='utf-8') as file:
    json.dump(pages_1st_process, file, ensure_ascii=False, indent=4)
with open(f"./image_model/json_data_fast_check_2_fixed.json", 'w', encoding='utf-8') as file:
    json.dump(pages_2nd_process, file, ensure_ascii=False, indent=4)
with open(f"./image_model/json_data_fact_checking.json", 'w', encoding='utf-8') as file:
    json.dump(pages_fact_checking, file, ensure_ascii=False, indent=4)